# DeepAR Model Training

In [2]:
import boto3
import sagemaker
import csv
import pandas as pd

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

## Get train/test dataframes from S3 

In [4]:
!aws s3 cp --recursive s3://aurelia-resort-data/model_train ./data

download: s3://aurelia-resort-data/model_train/test.csv to data/test.csv
download: s3://aurelia-resort-data/model_train/train.csv to data/train.csv


In [8]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [20]:
train.tail(5)

,Date,TotalAirlineTripstoDC,TotalAirlinePassengerstoDC,TotalAmericanTravelers,PercentofAmericanswhoTraveled,TotalTripsbyAmericans,num_stays,income_total
19,2021-08-01,401,1397805.0,2.603416e+08,81.851829,1.174912e+09,6244,15802.4
20,2021-09-01,446,1171253.0,2.613255e+08,81.973843,1.135523e+09,5304,15580.2
21,2021-10-01,487,1354033.0,2.602287e+08,81.601537,1.218701e+09,6618,15584.9
22,2021-11-01,472,1444907.0,2.596234e+08,81.463831,1.301882e+09,5586,15543.5
23,2021-12-01,485,1395610.0,2.594757e+08,82.110686,1.298152e+09,5147,15483.6


In [19]:
test.head(5)

,Date,TotalAirlineTripstoDC,TotalAirlinePassengerstoDC,TotalAmericanTravelers,PercentofAmericanswhoTraveled,TotalTripsbyAmericans,num_stays,income_total
0,2022-01-01,904,1893198.0,2.595847e+08,82.004716,1.243775e+09,4356,15137.7
1,2022-02-01,830,2139952.0,2.601098e+08,82.156684,1.210049e+09,4413,15125.6
2,2022-03-01,896,3059442.0,2.621167e+08,81.897555,1.218480e+09,7754,15064.1
3,2022-04-01,858,3272280.0,2.614628e+08,81.906942,1.279973e+09,9208,15055.2
4,2022-05-01,902,3415990.0,2.610540e+08,82.059921,1.305916e+09,10146,15036.4
